In [1]:
import numpy as np
import datajoint as dj
import trimesh
from collections import Counter
pinky = dj.create_virtual_module("pinky","microns_pinky")
schema = dj.schema("microns_pinky")

Connecting celiib@10.28.0.34:3306


In [5]:
@schema
class SpineValidationStatistics(dj.Computed):
    definition="""
    -> pinky.ProofreadLabelSpineWithInterestSlices
    ---
    n_auto_spine      : int unsigned                 # number of faces classified as spine,spine-head or spine-neck by auto-classifier
    n_auto_shaft      : int unsigned                 # number of faces classified as shaft by auto-classifier
    n_false_negatives :  int unsigned                # number of spines incorrectly autoclassified as shaft (merged spines would add 2,3,4...etc)
    n_false_positive  : int unsigned                 # number of sections of the shaft that were incorrectly classified as spine
    true_positives    : int unsigned                 # number of spines correctly classified as spines
    n_merged_spines   : int unsigned                 # total number of additional spines that resulted from being merged with others
    perc_spines_with_merges=null : decimal(6,5)           # percentage of the true positive count that also had other spines merged onto it
    n_spines_with_overspined: int unsigned           # total number out of the true positives that had overspining occuring on their labeling
    average_percentage_overspined=null: decimal(6,5)      # for those that had overspining, the average face percentage that were labeled as overspined
    avg_spine_size=null    : decimal(10,5)                # average number of faces for the true positive spines
    
    """
    
    key_source = pinky.ProofreadLabelSpineWithInterestSlices() & dict(author_proofreader="js48",status="complete")
    
    def make(self,key):
        print("key  = " + str(key))
        significant_size = 50
        search_key = dict(author_proofreader="js48",status="complete")
        search_dict = dict(search_key,segmentation=key["segmentation"],
                          segment_id=key["segment_id"],
                          compartment_type=key["compartment_type"],
                          component_index=key["component_index"])
        
        # get the mesh and the original and proofread labels
        """
        way to get the entire mesh:
        1) get the id, compartment type and component index, starting index, number of faces, and triangle indices
        from the validation proofing table
        2) Get the whole mesh from: 
        a. pinky.PymeshfixDecimatedExcitatoryStitchedMesh
        b. pinky.Decimation35OrphanStitched

        3) Use the triangle indices to restrict to the actual portion of submesh

        """

        #Example
        verts,faces = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh() & search_dict).fetch1("vertices","triangles")

        #get compartment verts and faces
        verts_comp,faces_comp = (pinky.CompartmentFinal.ComponentFinal & search_dict).fetch1("vertex_indices","triangle_indices")

        #build the trimesh object
        new_mesh = trimesh.Trimesh(verts,faces)
        #new_mesh.show()
        compartment_mesh = new_mesh.submesh([faces_comp],append=True)

        #now get the submesh according to the triangle indices
        tri_indices = (pinky.ProofreadLabelSpineWithInterestSlices & search_dict).fetch1("triangle_indices")
        random_mesh = compartment_mesh.submesh([tri_indices],append=True)
        #random_mesh.show()

        # how to get the automatic labels: 
        verts_labels,triangles_labels = (pinky.ComponentLabelFinal & search_dict).fetch1("labeled_vertices","labeled_triangles")

        verts_labels = verts_labels.astype("int")
        triangles_labels = triangles_labels.astype("int")


        #get the new triangle lables based on the submesh indexes
        auto_labels = triangles_labels[tri_indices]
        auto_labels[auto_labels == 0] = 17
        auto_labels[auto_labels == 12] = 14
        auto_labels[auto_labels == 13] = 14
        auto_labels[auto_labels == 15] = 14
        auto_labels = auto_labels
        auto_labels

        #how to get the proofread labels
        tri_labels_proofread = (pinky.ProofreadLabelSpineWithInterestSlices & search_dict).fetch1("triangles")
        tri_labels_proofread
        
        #get rid of all of the error and no_label sections: 
        labels_dict = dict(
        spine_1=12,
            shaft=2,
            shaft_over_spined=18,
            shaft_3=5,
            remove_label=0,
            error=10,
            spine_2=3,
            spine_3=4,
            spine_4=6,
            spine_5=9,
            all_one_spine=11,
            auto_shaft=17,
            auto_spine=14,
            spine_head_manual=13,
            spine_neck_manual=15,
            label_removed=1
        )

        labels_dict_flipped = dict([(v,k) for k,v in labels_dict.items()])
        labels_dict_flipped
        
        """
        1) Handle the all-one-spine pieces: get the submesh and split pieces, count the number of pieces as false negatives (SIGNIFICANT SIZED) 
         Eliminate any mesh pieces that are error or no label
        """
        all_one_spine_n_faces = 0
        all_one_spine_n_bodies = 0

        x = np.where(tri_labels_proofread == labels_dict["all_one_spine"])[0]
        if len(x) > 0:
            all_one_spine_n_faces = len(x)
            #get the submesh
            all_one_spine_mesh = random_mesh.submesh([x],append=True)
            #get the number of splits
            all_one_spine_mesh_split = all_one_spine_mesh.split(only_watertight=False)
            all_one_spine_n_bodies = len(all_one_spine_mesh_split)

        print("all_one_spine_n_bodies = " + str(all_one_spine_n_bodies)) 
        print("all_one_spine_n_faces = " + str(all_one_spine_n_faces))

        print("Done finding all in one spines")

        
        """
        For Shaft: 
        1) From original mesh extract the indices that were auto-shaft (get the number of faces in submesh to store later)
        2) Use the shaft indices to extract the new proofread labels for the autoshaft

        --- set size threshold for the spine counting (50)
        4) *** DON'T HAVE TO DO ANYMORE **** Get all the all_one_spine label indices --> create submesh from those. Count the number of all_one_spine as a missed spine
        5) Get the submesh of anything that is spine_1,spine_2,spine_3,spine_4,spine_5
        6) Split the submesh into disjointed parts (using connected_components to get the indices)
        7) Loop through the list (with indices)
        - if not meet desired threshold then put as 0
        - count the number of different spine labels that exist
        - store in array
        8) Sum the array and add on the all-one-spine number to count the total number missed
        """

        #extract the indices that were auto-shaft

        auto_shaft_indices = np.where(auto_labels==labels_dict["auto_shaft"])[0]
        #random_mesh_auto_shaft_indices = clean_indices[auto_shaft_indices]
        n_auto_shaft = len(auto_shaft_indices)
        n_auto_shaft

        #extract teh proofread shaft labels
        tri_labels_proofread_shaft = tri_labels_proofread[auto_shaft_indices]
        total_Counter = Counter(tri_labels_proofread_shaft)
        shaft_counts = dict([(labels_dict_flipped[k],total_Counter[k]) for k in dict(total_Counter).keys() ])
        print("After extracting the shaft after cleaning Distribution is:" + str(shaft_counts) )


        #get the indexes of all of the spines
        spine_numbers = [labels_dict["spine_" + str(k)] for k in range(1,6)] 
        print(spine_numbers)
        # np.where(tri_labels_proofread_clean_shaft 
        spine_indexes = np.array([])

        for iy in spine_numbers: 
            print("Working on spine "+ str(labels_dict_flipped[iy]))
            spine_indexes = np.hstack((spine_indexes, np.where(tri_labels_proofread_shaft==iy)[0]))
            print(f"Len of spine indexes = {len(spine_indexes)}")
        spine_indexes = spine_indexes.astype("int")
        #check to see if there were any spines: 
        total_spine_count_list = []

        if (spine_indexes.any()):
            #get the spines only submesh
            spine_submesh = random_mesh.submesh([auto_shaft_indices[spine_indexes]],append=True)
            #divide the spine submesh into disconnected portions
            spine_disconnected_groups = trimesh.graph.connected_components(edges=spine_submesh.face_adjacency,
                                          nodes=np.arange(len(spine_submesh.faces)),
                                          min_len=1,
                                          engine=None)
            #REMINDER: these indices correspond to the spine mesh vertices (which are spine indexes of auto_shaft of random mesh)
            print(len(spine_disconnected_groups))

            if len(spine_disconnected_groups)>0:
                for i,spine_group in enumerate(spine_disconnected_groups):
                    if len(spine_group) < significant_size:
                        print("Size of group too small for a spine")
                        continue
                    print("Working on shaft spine #" + str(i))
                    #iterate through and see what spine labels are present
                    original_labels_for_spine = tri_labels_proofread_shaft[spine_indexes[spine_group]]
                    print(Counter(original_labels_for_spine))

                    #see how many spine labels are in the proofread version labels for this spine
                    current_spine_count = 0
                    for iy in spine_numbers: 

                        #print("Working on spine "+ str(labels_dict_flipped[iy]))
                        if len(np.where(original_labels_for_spine==iy)[0]) > 0: 
                            print(f"Found {iy} in spine group")
                            current_spine_count += 1

                    total_spine_count_list.append(current_spine_count)

        print(total_spine_count_list)
        total_false_negative = sum(total_spine_count_list)
        print("total_false_negative = " + str(total_false_negative))
        
        """
        For spines: 
        1) From original mesh extract indices that were auto-spine (get the number of faces in submesh)
        2) Use auto-spine indices to extract the proofread labels for autoshaft
        --- set size threshold
        4) Dividee the submesh into all disjointed pieces (count the number of pieces and save off)
        5) Calculate the number of shaft and over-spine and percentage there are (WILL CALCULATE THIS AS ITERATING THROUGH)
        5) Count the number of pieces that are 70% or greater as false positives and remove those indices from the list (FALSE POSITIVES)
        6) For each piece still left in the list
        - count the number of different spine labels that exist (if only auto-spine then tally that as perfect)
        - store in array
        6) Sum up the array and tally up the number that got perfect
        """

        #extract the indices that were auto-shaft

        auto_spine_indices = np.where(auto_labels==labels_dict["auto_spine"])[0]
        #random_mesh_auto_shaft_indices = clean_indices[auto_shaft_indices]
        n_auto_spine = len(auto_spine_indices)
        print("n_auto_spine = " + str(n_auto_spine))

        #extract teh proofread spine labels
        tri_labels_proofread_spine = tri_labels_proofread[auto_spine_indices]
        total_Counter = Counter(tri_labels_proofread_spine)
        spine_counts = dict([(labels_dict_flipped[k],total_Counter[k]) for k in dict(total_Counter).keys() ])
        print("After extracting the shaft after cleaning Distribution is:" + str(spine_counts) )


        #get the indexes of all of the spines
        spine_numbers = [labels_dict["spine_" + str(k)] for k in range(1,6)]
        spine_numbers_and_spine_overspine = spine_numbers + [labels_dict["auto_spine"],labels_dict["shaft_over_spined"],
                                                             labels_dict["shaft"],
                                                             labels_dict["shaft_3"]
                                                            ]
        #print(spine_numbers_and_spine_overspine)
        # np.where(tri_labels_proofread_clean_shaft 
        spine_indexes = np.array([])

        for iy in spine_numbers_and_spine_overspine: 
            #print("Working on spine "+ str(labels_dict_flipped[iy]))
            spine_indexes = np.hstack((spine_indexes, np.where(tri_labels_proofread_spine==iy)[0]))
            print(f"Len of spine indexes = {len(spine_indexes)}")
        spine_indexes = spine_indexes.astype("int")
        #check to see if there were any spines: 
        additional_spine_count_list = []
        overspined_list = []
        overspined_list_perc = []
        spine_size_list = []
        false_positive_count = 0
        auto_distinct_spines = 0

        #
        if (spine_indexes.any()):
            #get the spines only submesh
            spine_submesh = random_mesh.submesh([auto_spine_indices[spine_indexes]],append=True)
            #divide the spine submesh into disconnected portions
            spine_disconnected_groups = trimesh.graph.connected_components(edges=spine_submesh.face_adjacency,
                                          nodes=np.arange(len(spine_submesh.faces)),
                                          min_len=1,
                                          engine=None)
            #REMINDER: these indices correspond to the spine mesh vertices (which are spine indexes of auto_shaft of random mesh)
            #print(len(spine_disconnected_groups))


            if len(spine_disconnected_groups)>0:
                auto_distinct_spines = len(spine_disconnected_groups)
                for i,spine_group in enumerate(spine_disconnected_groups):

                    if len(spine_group) < significant_size:
                        print("Size of group too small for a spine")
                        continue




                    #print("Working on shaft spine #" + str(i))
                    #iterate through and see what spine labels are present
                    original_labels_for_spine = tri_labels_proofread_spine[spine_indexes[spine_group]]
                    original_labels_counter = Counter(original_labels_for_spine)
                    #print(original_labels_counter)

                    #get the number of shaft faces
                    current_shaft_counter = 0
                    if labels_dict["shaft_3"] in original_labels_counter.keys():
                        current_shaft_counter += original_labels_counter[labels_dict["shaft_3"]]
                    if labels_dict["shaft"] in original_labels_counter.keys():
                        current_shaft_counter += original_labels_counter[labels_dict["shaft"]]

                    #count the amount it was labeled as a shaft
                    if current_shaft_counter/len(spine_group) > 0.70:
                        print("Found false positive for spine " + str(i))
                        false_positive_count += 1
                        continue

                    spine_size_list.append(len(spine_group))


                    if labels_dict["shaft_over_spined"] in original_labels_counter.keys(): 
                        overspined_list.append(original_labels_counter[labels_dict["shaft_over_spined"]])
                        overspined_list_perc.append(original_labels_counter[labels_dict["shaft_over_spined"]]/len(spine_group))
                    else:
                        overspined_list.append(0)
                        overspined_list_perc.append(0)

                    #see how many spine labels are in the proofread version labels for this spine
                    current_spine_count = 0
                    for iy in spine_numbers: 

                        #print("Working on spine "+ str(labels_dict_flipped[iy]))
                        if len(np.where(original_labels_for_spine==iy)[0]) > 0: 
                            #print(f"Found {iy} in spine group")
                            current_spine_count += 1

                    additional_spine_count_list.append(current_spine_count)

#         print("additional_spine_count_list = " + str(additional_spine_count_list))

#         print("overspined_list = " + str(overspined_list))

#         print("overspined_list_perc = " + str(overspined_list_perc))

#         print("false_positive_count = " + str(false_positive_count))

#         print("auto_distinct_spines = " + str(auto_distinct_spines))

#         print("spine_size_list = " + str(spine_size_list))
                                                               
        #the final stats that wanted
        #print("additional_spine_count_list = " + str(additional_spine_count_list))

        #stats want to keep from the shaft

        #stats we want to keep from the spine

        total_false_negative = sum(total_spine_count_list)
        print("total_false_negative = " + str(total_false_negative))

        n_merged_spines = sum(additional_spine_count_list)
        print("n_merged_spines = " + str(n_merged_spines))
        perc_spines_with_merges = len(np.where(np.array(additional_spine_count_list)>0)[0])/len(additional_spine_count_list)
        print("perc_spines_with_merges = " + str(perc_spines_with_merges))

        #print("overspined_list = " + str(overspined_list))
        overspined_faces = np.where(np.array(overspined_list)>0)[0]
        n_spines_with_overspined = len(overspined_faces)
        print("n_spines_with_overspined = " + str(n_spines_with_overspined))

        #print("overspined_list_perc = " + str(overspined_list_perc))
        #average percentage of overspined 
        average_percentage_overspined = np.mean(np.array(overspined_list_perc)[overspined_faces])
        print("average_percentage_overspined = " + str(average_percentage_overspined))

        print("false_positive_count = " + str(false_positive_count))

        print("auto_distinct_spines = " + str(auto_distinct_spines))
        avg_spine_size = np.mean(spine_size_list)
        print("avg_spine_size = " + str(avg_spine_size))

        print('n_auto_spine = ' + str(n_auto_spine))
        print('n_auto_shaft = ' + str(n_auto_shaft))

        """
        n_auto_spine      : int unsigned                 # number of faces classified as spine,spine-head or spine-neck by auto-classifier
        n_auto_shaft      : int unsigned                 # number of faces classified as shaft by auto-classifier
        n_false_negatives :  int unsigned                # number of spines incorrectly autoclassified as shaft (merged spines would add 2,3,4...etc)
        n_false_positive  : int unsigned                 # number of sections of the shaft that were incorrectly classified as spine
        true_positives    : int unsigned                 # number of spines correctly classified as spines
        n_merged_spines   : int unsigned                 # total number of additional spines that resulted from being merged with others
        perc_spines_with_merges : decimal(6,5)           # percentage of the true positive count that also had other spines merged onto it
        n_spines_with_overspined: int unsigned           # total number out of the true positives that had overspining occuring on their labeling
        average_percentage_overspined: decimal(6,5)      # for those that had overspining, the average face percentage that were labeled as overspined
        avg_spine_size    : decimal(10,5)                # average number of faces for the true positive spines
        """
        
        
        #insert into the table
        new_dict = dict(key,
                       n_auto_spine=n_auto_spine,
                       n_auto_shaft=n_auto_shaft,
                       n_false_negatives=total_false_negative,
                       n_false_positive=false_positive_count,
                       true_positives=auto_distinct_spines,
                       n_merged_spines=n_merged_spines,
                       perc_spines_with_merges=perc_spines_with_merges,
                       n_spines_with_overspined=n_spines_with_overspined,
                       average_percentage_overspined=average_percentage_overspined,
                       avg_spine_size=avg_spine_size)

        self.insert1(new_dict,skip_duplicates=True)


In [6]:
#(schema.jobs & "table_name='__spine_validation_statistics'").delete()

In [7]:
# key  = {'segmentation': 3, 'segment_id': 648518346341371119, 'decimation_ratio': 0.35, 'compartment_type': 'Oblique', 'component_index': 0}

In [8]:
import time
start_time = time.time()
SpineValidationStatistics.populate(reserve_jobs=True)
print(f"Total time = {time.time() - start_time}")

key  = {'segmentation': 3, 'segment_id': 648518346349498116, 'decimation_ratio': Decimal('0.35'), 'compartment_type': 'Basal', 'component_index': 3, 'author_original': 'computer_Auto', 'author_proofreader': 'js48', 'starting_index': 11805, 'n_triangles': 30000, 'date_time': datetime.datetime(2019, 8, 12, 16, 7, 52)}
all_one_spine_n_bodies = 0
all_one_spine_n_faces = 0
Done finding all in one spines
After extracting the shaft after cleaning Distribution is:{'auto_shaft': 11402, 'spine_1': 1861}
[12, 3, 4, 6, 9]
Working on spine spine_1
Len of spine indexes = 1861
Working on spine spine_2
Len of spine indexes = 1861
Working on spine spine_3
Len of spine indexes = 1861
Working on spine spine_4
Len of spine indexes = 1861
Working on spine spine_5
Len of spine indexes = 1861
5
Working on shaft spine #0
Counter({12: 308})
Found 12 in spine group
Working on shaft spine #1
Counter({12: 337})
Found 12 in spine group
Working on shaft spine #2
Counter({12: 464})
Found 12 in spine group
Working on

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


key  = {'segmentation': 3, 'segment_id': 648518346349498632, 'decimation_ratio': Decimal('0.35'), 'compartment_type': 'Oblique', 'component_index': 3, 'author_original': 'computer_Auto', 'author_proofreader': 'js48', 'starting_index': 6191, 'n_triangles': 30000, 'date_time': datetime.datetime(2019, 8, 12, 16, 19, 7)}
all_one_spine_n_bodies = 0
all_one_spine_n_faces = 0
Done finding all in one spines
After extracting the shaft after cleaning Distribution is:{'auto_shaft': 10783, 'label_removed': 136, 'shaft_over_spined': 3, 'spine_1': 682}
[12, 3, 4, 6, 9]
Working on spine spine_1
Len of spine indexes = 682
Working on spine spine_2
Len of spine indexes = 682
Working on spine spine_3
Len of spine indexes = 682
Working on spine spine_4
Len of spine indexes = 682
Working on spine spine_5
Len of spine indexes = 682
3
Working on shaft spine #0
Counter({12: 425})
Found 12 in spine group
Working on shaft spine #1
Counter({12: 120})
Found 12 in spine group
Working on shaft spine #2
Counter({12:

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


key  = {'segmentation': 3, 'segment_id': 648518346349499085, 'decimation_ratio': Decimal('0.35'), 'compartment_type': 'Basal', 'component_index': 5, 'author_original': 'computer_Auto', 'author_proofreader': 'js48', 'starting_index': 59041, 'n_triangles': 30000, 'date_time': datetime.datetime(2019, 8, 14, 17, 53, 5)}
all_one_spine_n_bodies = 0
all_one_spine_n_faces = 0
Done finding all in one spines
After extracting the shaft after cleaning Distribution is:{'auto_shaft': 16734, 'spine_1': 1763, 'label_removed': 100}
[12, 3, 4, 6, 9]
Working on spine spine_1
Len of spine indexes = 1763
Working on spine spine_2
Len of spine indexes = 1763
Working on spine spine_3
Len of spine indexes = 1763
Working on spine spine_4
Len of spine indexes = 1763
Working on spine spine_5
Len of spine indexes = 1763
7
Working on shaft spine #0
Counter({12: 213})
Found 12 in spine group
Working on shaft spine #1
Counter({12: 359})
Found 12 in spine group
Size of group too small for a spine
Working on shaft spin

...

key  = {'segmentation': 3, 'segment_id': 648518346349511866, 'decimation_ratio': Decimal('0.35'), 'compartment_type': 'Oblique', 'component_index': 0, 'author_original': 'computer_Auto', 'author_proofreader': 'js48', 'starting_index': 8090, 'n_triangles': 30000, 'date_time': datetime.datetime(2019, 8, 12, 15, 50, 52)}
all_one_spine_n_bodies = 0
all_one_spine_n_faces = 0
Done finding all in one spines
After extracting the shaft after cleaning Distribution is:{'auto_shaft': 11086, 'spine_1': 774, 'label_removed': 83, 'shaft_over_spined': 1}
[12, 3, 4, 6, 9]
Working on spine spine_1
Len of spine indexes = 774
Working on spine spine_2
Len of spine indexes = 774
Working on spine spine_3
Len of spine indexes = 774
Working on spine spine_4
Len of spine indexes = 774
Working on spine spine_5
Len of spine indexes = 774
5
Working on shaft spine #0
Counter({12: 279})
Found 12 in spine group
Working on shaft spine #1
Counter({12: 310})
Found 12 in spine group
Size of group too small for a spine
Wo

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


key  = {'segmentation': 3, 'segment_id': 648518346349512186, 'decimation_ratio': Decimal('0.35'), 'compartment_type': 'Basal', 'component_index': 3, 'author_original': 'computer_Auto', 'author_proofreader': 'js48', 'starting_index': 3515, 'n_triangles': 11044, 'date_time': datetime.datetime(2019, 8, 20, 15, 23, 50)}
all_one_spine_n_bodies = 0
all_one_spine_n_faces = 0
Done finding all in one spines
After extracting the shaft after cleaning Distribution is:{'auto_shaft': 9297, 'shaft_over_spined': 12}
[12, 3, 4, 6, 9]
Working on spine spine_1
Len of spine indexes = 0
Working on spine spine_2
Len of spine indexes = 0
Working on spine spine_3
Len of spine indexes = 0
Working on spine spine_4
Len of spine indexes = 0
Working on spine spine_5
Len of spine indexes = 0
[]
total_false_negative = 0
n_auto_spine = 1735
After extracting the shaft after cleaning Distribution is:{'auto_spine': 1596, 'shaft_over_spined': 139}
Len of spine indexes = 0
Len of spine indexes = 0
Len of spine indexes = 0

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


key  = {'segmentation': 3, 'segment_id': 648518346349513135, 'decimation_ratio': Decimal('0.35'), 'compartment_type': 'Oblique', 'component_index': 0, 'author_original': 'computer_Auto', 'author_proofreader': 'js48', 'starting_index': 22919, 'n_triangles': 30000, 'date_time': datetime.datetime(2019, 8, 12, 15, 49, 45)}
all_one_spine_n_bodies = 0
all_one_spine_n_faces = 0
Done finding all in one spines
After extracting the shaft after cleaning Distribution is:{'auto_shaft': 12110, 'spine_1': 1605, 'label_removed': 59}
[12, 3, 4, 6, 9]
Working on spine spine_1
Len of spine indexes = 1605
Working on spine spine_2
Len of spine indexes = 1605
Working on spine spine_3
Len of spine indexes = 1605
Working on spine spine_4
Len of spine indexes = 1605
Working on spine spine_5
Len of spine indexes = 1605
9
Working on shaft spine #0
Counter({12: 300})
Found 12 in spine group
Working on shaft spine #1
Counter({12: 322})
Found 12 in spine group
Size of group too small for a spine
Working on shaft sp